In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, Dense, Dropout, Flatten, MaxPooling1D

In [2]:
# Load data
df = pd.read_csv("space_mountain_with_holiday_weather_lag_suite.csv")

# Parse date
df['Date'] = pd.to_datetime(df['Date'], format='%m/%d/%y')
df['Datetime'] = pd.to_datetime(df['Date'].astype(str) + ' ' + df['Time'])
df.sort_values('Datetime', inplace=True)

# One-hot encode Day of Week
df = pd.get_dummies(df, columns=['Day of Week'], drop_first=True)

# Drop unused columns
df.drop(columns=['Date', 'Time', 'Datetime'], inplace=True)

# Scale all features
scaler = StandardScaler()
scaled = scaler.fit_transform(df)
scaled_df = pd.DataFrame(scaled, columns=df.columns)

In [3]:
SEQUENCE_LENGTH = 12
X, y = [], []
features = scaled_df.drop(columns=['Wait Time']).values
wait_values = scaled_df['Wait Time'].values

for i in range(SEQUENCE_LENGTH, len(scaled_df)):
    X.append(features[i-SEQUENCE_LENGTH:i])
    y.append(wait_values[i])

X = np.array(X)
y = np.array(y)

# Train/test split
split_index = int(0.8 * len(X))
X_train, X_test = X[:split_index], X[split_index:]
y_train, y_test = y[:split_index], y[split_index:]

In [4]:
model = Sequential()
model.add(Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(X_train.shape[1], X_train.shape[2])))
model.add(MaxPooling1D(pool_size=2))
model.add(Dropout(0.3))
model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dense(1))

model.compile(optimizer='adam', loss='mse')
model.summary()

# Train
history = model.fit(X_train, y_train, epochs=20, batch_size=32, validation_split=0.1, verbose=1)

/Users/jeremypogue/miniforge3/lib/python3.12/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d (Conv1D)                 │ (None, 10, 64)         │         3,136 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d (MaxPooling1D)    │ (None, 5, 64)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 5, 64)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 320)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 64)             │        20,544 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 23,745 (92.75 KB)

 Trainable params: 23,745 (92.75 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/20
1085/1085 ━━━━━━━━━━━━━━━━━━━━ 1s 880us/step - loss: 0.8942 - val_loss: 0.6104
Epoch 2/20
1085/1085 ━━━━━━━━━━━━━━━━━━━━ 1s 915us/step - loss: 0.7564 - val_loss: 0.5894
Epoch 3/20
1085/1085 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 0.7239 - val_loss: 0.5833
Epoch 4/20
1085/1085 ━━━━━━━━━━━━━━━━━━━━ 1s 924us/step - loss: 0.7187 - val_loss: 0.5931
Epoch 5/20
1085/1085 ━━━━━━━━━━━━━━━━━━━━ 1s 865us/step - loss: 0.7014 - val_loss: 0.5962
Epoch 6/20
1085/1085 ━━━━━━━━━━━━━━━━━━━━ 1s 872us/step - loss: 0.6921 - val_loss: 0.6134
Epoch 7/20
1085/1085 ━━━━━━━━━━━━━━━━━━━━ 1s 841us/step - loss: 0.6683 - val_loss: 0.5894
Epoch 8/20
1085/1085 ━━━━━━━━━━━━━━━━━━━━ 1s 897us/step - loss: 0.6500 - val_loss: 0.5838
Epoch 9/20
1085/1085 ━━━━━━━━━━━━━━━━━━━━ 1s 860us/step - loss: 0.6519 - val_loss: 0.6050
Epoch 10/20
1085/1085 ━━━━━━━━━━━━━━━━━━━━ 1s 841us/step - loss: 0.6447 - val_loss: 0.5874
Epoch 11/20
1085/1085 ━━━━━━━━━━━━━━━━━━━━ 1s 867us/step - loss: 0.6430 - val_loss: 0.6040
Epoch 12/2

In [5]:
# Predict
y_pred = model.predict(X_test)

# Inverse transform to real wait time
wait_scaler = StandardScaler()
wait_scaler.fit(df[['Wait Time']])
y_pred_unscaled = wait_scaler.inverse_transform(y_pred)
y_test_unscaled = wait_scaler.inverse_transform(y_test.reshape(-1, 1))

real_rmse = np.sqrt(mean_squared_error(y_test_unscaled, y_pred_unscaled))
print(f"Conv1D Unscaled RMSE (in minutes): {real_rmse:.2f}")

302/302 ━━━━━━━━━━━━━━━━━━━━ 0s 407us/step
Conv1D Unscaled RMSE (in minutes): 18.22
